In [ ]:
# 1. Install Transformers (if not already installed)
!pip install transformers

In [ ]:
!pip install transformers datasets accelerate peft -q

In [ ]:
pip install --upgrade pip

#Importing required libraries

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
!pip install transformers accelerate peft

In [ ]:
import pandas as pd
import numpy as np

print("Pandas version:", pd.__version__)
print("Numpy version:", np.__version__)

In [ ]:
import os
print(os.path.exists("/kaggle/input/clinical-reasoning-dataset/train.csv"))


In [ ]:
import pandas as pd

train_df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/train.csv")
test_df = pd.read_csv("/kaggle/input/clinical-reasoning-dataset/test.csv")

In [ ]:
train_df.info()

In [ ]:
train_df.columns

In [ ]:

train_df.drop('Master_Index', axis=1, inplace=True) 

In [ ]:
train_df.isnull().sum()

In [ ]:
import seaborn as sns
sns.histplot(data=train_df, x='Years of Experience')

In [ ]:
median_exp = train_df["Years of Experience"].median()
train_df["Years of Experience"] = train_df["Years of Experience"].fillna(median_exp)

In [ ]:
train_df.isnull().sum()

In [ ]:
def build_context(row):
    return (
        f"County: {row['County']}, "
        f"Health level: {row['Health level']}, "
        f"Years of Experience: {row['Years of Experience']}, "
        f"Nursing Competency: {row['Nursing Competency']}, "
        f"Clinical Panel: {row['Clinical Panel']}"
    )

train_df["Context"] = train_df.apply(build_context, axis=1)

# Construct the final SLM input prompt
train_df["Input"] = "Context: " + train_df["Context"] + "\nPrompt: " + train_df["Prompt"]

# Define target output
train_df["Target"] = train_df["Clinician"]

# Drop rows where Target is missing
train_df = train_df[train_df["Target"].notna()]

# Save for training
train_df[["Input", "Target"]].to_csv("formatted_train_data.csv", index=False)

# example prompt

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_id = "google/flan-t5-small"

# Load model
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

# Define prompt
prompt = """
Context: County: Kisumu, Health level: Level 3, Years of Experience: 10, Nursing Competency: High
Prompt: A 22-year-old woman presents with fever, abdominal pain, and delayed menstruation.
What would the clinician do?
"""

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
outputs = model.generate(**inputs, max_new_tokens=100)

# Decode and print
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
!pip install transformers datasets accelerate

import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)



In [ ]:
# Step 1: Load your formatted dataset
# Check first if the file exists
import os

if os.path.exists("/kaggle/input/input-output-formatted-csv/formatted_train_data.csv"):
    train_df_clean = pd.read_csv("/kaggle/input/input-output-formatted-csv/formatted_train_data.csv")
    print("Loaded successfully.")
else:
    print("File not found. Please ensure it was written in a previous step.")



In [ ]:
from datasets import Dataset

# Convert your cleaned DataFrame to a Hugging Face Dataset
train_dataset = Dataset.from_pandas(train_df_clean[["Input", "Target"]])

In [ ]:
# Step 2: Load FLAN-T5 tokenizer and model
model_name = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)



In [ ]:
!pip install transformers datasets accelerate

import pandas as pd
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Trainer,
    TrainingArguments,
)

In [ ]:
max_input_length = 512
max_target_length = 128

def preprocess(example):
    model_input = tokenizer(
        example["Input"],
        max_length=max_input_length,
        truncation=True,
        padding="max_length",
    )
    labels = tokenizer(
        example["Target"],
        max_length=max_target_length,
        truncation=True,
        padding="max_length",
    )
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = train_dataset.map(preprocess, batched=True)


In [ ]:
import transformers
print(transformers.__version__)


In [ ]:
from transformers import TrainingArguments

In [ ]:
!pip install -U transformers
from transformers import TrainingArguments

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./flan-t5-clinician",
    evaluation_strategy="no",  # or "steps" / "epoch"
    per_device_train_batch_size=8,
    num_train_epochs=3,
    save_total_limit=1,
    save_steps=500,
    logging_dir="./logs",
    logging_steps=100,
    fp16=False,
    report_to="none"
)


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./test-model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    evaluation_strategy="epoch",  # <-- should not error
)


In [ ]:
from transformers import TrainingArguments
print(TrainingArguments.__module__)
print(TrainingArguments.__doc__)

In [ ]:
!pip install -U transformers --quiet

In [ ]:
from transformers import TrainingArguments

print(TrainingArguments.__module__)


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./test-model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
)


In [ ]:
import os
print(os.listdir())

In [ ]:
import transformers
print(transformers.__file__)


In [ ]:
!pip install -U transformers --quiet

from transformers import TrainingArguments

print(TrainingArguments.__module__)  # Should say transformers.training_args


In [ ]:
args = TrainingArguments(
    output_dir="./test-model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    evaluation_strategy="epoch",
)


In [ ]:
print(transformers.__file__)

In [ ]:
!pip uninstall -y transformers


In [ ]:
!pip install transformers==4.51.3 --force-reinstall --no-cache-dir


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="./test-model",
    per_device_train_batch_size=2,
    num_train_epochs=1,
    evaluation_strategy="epoch",  # <-- this must now work
)

print("✅ Success!")
